In [1]:
import pandas as pd
import re

In [2]:
from re import sub
def snake_case(s):
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    s.replace('-', ' '))).split()).lower()

In [3]:
results_file = pd.ExcelFile('results/results.xlsx')
results_file

In [4]:
q_file = pd.read_excel('results/questionnaires.xlsx')
q_file

,request_id,keyword,question,generated_by,questionnaire
0,500,Houseplants,Are you a fan of houseplants?,gpt3,3
1,785,Kate Bush,Are you a fan of Kate Bush?,human,3
2,663,Poetry,Do you like Poetry?,human,3
3,120,Surfing,What is your opinion on surfing?,gpt3,3
4,374,Pizza,Pizza or tacos?,gpt3,3
...,...,...,...,...,...
295,163,Ghosts,Do you believe in ghosts?,gpt3,10
296,68,Exams,What is your opinion on exams?,gpt3,10
297,684,Influencers,Is influencer a real job?,human,10
298,737,Batgirl,Did you ever see a Batgirl comic?,human,10


In [12]:
q_results_dict: dict[int, pd.DataFrame] = {}
num_respondants = 0

for sheet_name in results_file.sheet_names:
    q_id = int(re.search(r'form_responses_(\d+)', snake_case(sheet_name)).group(1))
    
    df = results_file.parse(sheet_name)
    df.columns = df.columns.str.strip()
    df = df.drop(['Timestamp', 'Score'], axis=1)
    df  = df.T
    df.index.name = 'question'
    df = df.reset_index()
    
    questions = df['question']
    responses = df.columns.values.tolist()
    responses.remove('question')
    response_df = pd.DataFrame()

    for response_value in responses:
        num_respondants += 1
        new = pd.DataFrame()
        new['response'] = df[response_value].map({'Human': 'human', 'Computer': 'gpt3'})
        new['question'] = df['question']
        new['respondant_id'] = num_respondants
        response_df = pd.concat([response_df, new])
    
    q_results_dict[q_id] = response_df
    print(response_df)

   response                                           question  respondant_id
0     human               Do you agree with the Brexit result?              1
1     human  Are you in favor of exams starting at a specif...              1
2     human  Who do you think will win the Premier League t...              1
3      gpt3  Do you think that women's football is a sport ...              1
4      gpt3                What is your opinion on Kanye West?              1
5      gpt3  Do you think 'cancelling' is an appropriate wa...              1
6      gpt3               What is your opinion on influencers?              1
7      gpt3                        Kanye West or Taylor Swift?              1
8      gpt3             What is your opinion on thunderstorms?              1
9      gpt3               What is your opinion on music taste?              1
10     gpt3  Do you think that wind turbines are a good way...              1
11    human    Do you prefer working at the office or at home?  